In [1]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from datetime import datetime
from functools import reduce
import json
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [2]:
#Define presto credentials
presto_host = 'presto-gateway.serving.data.plectrum.dev'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [3]:
#Load the hexes in zone and holdout
hexes = pd.read_excel('auto_segments_bangalore.xlsx', sheet_name = 'hsr_new_zone')
hexes = hexes.rename(columns = {'uid':'pickup_location_hex_8', 'segment':'Segment'})
hyderabad_new_zone = hexes['pickup_location_hex_8'].unique().tolist()
hexes.head()

,pickup_location_hex_8,Segment,Label
0,88618924bbfffff,in-low,Holdout
1,886189260bfffff,in-low,Holdout
2,8860145913fffff,in-low,Holdout
3,88618924b3fffff,in-low,Holdout
4,8860145823fffff,in-low,Holdout


In [4]:
print(hyderabad_new_zone)

['88618924bbfffff', '886189260bfffff', '8860145913fffff', '88618924b3fffff', '8860145823fffff', '8860169659fffff', '886014595dfffff', '8860145a07fffff', '8860145847fffff', '8860169607fffff', '8861892c81fffff', '88601451e1fffff', '8861892687fffff', '88618921e3fffff', '8861892187fffff', '88601451e5fffff', '8860145acdfffff', '88618920ebfffff', '886189203dfffff', '8860145a83fffff', '8860145a87fffff', '886189208dfffff', '8860169645fffff', '88601696dbfffff', '8861892debfffff', '8861892495fffff', '8860145125fffff', '8861892c3bfffff', '88618921c5fffff', '8860169621fffff', '8860169619fffff', '8860145aadfffff', '8860145921fffff', '88618934e3fffff', '886014c905fffff', '8860145907fffff', '88618921cdfffff', '8861892281fffff', '8860145101fffff', '8860145ab5fffff', '8861892117fffff', '8861893495fffff', '88601690c7fffff', '886016968dfffff', '8860145995fffff', '886189228dfffff', '886016944bfffff', '88618934b1fffff', '8861892f65fffff', '8860169687fffff', '8860169743fffff', '8860145115fffff', '8861892691

In [15]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, city_name, hhmmss, hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, eta, map_riders, pickup_location_hex_8
from orders.order_logs_snapshot
where yyyymmdd in ('20230424', '20230425', '20230426', '20230515', '20230516', '20230517')
and city_name in ('Bangalore')
and service_obj_service_name = 'Auto'
and pickup_location_hex_8 in ('88618924bbfffff', '886189260bfffff', '8860145913fffff', '88618924b3fffff', '8860145823fffff', '8860169659fffff', '886014595dfffff',
 '8860145a07fffff', '8860145847fffff', '8860169607fffff', '8861892c81fffff', '88601451e1fffff', '8861892687fffff', '88618921e3fffff', '8861892187fffff',
   '88601451e5fffff', '8860145acdfffff', '88618920ebfffff', '886189203dfffff', '8860145a83fffff', '8860145a87fffff', '886189208dfffff', '8860169645fffff',
     '88601696dbfffff', '8861892debfffff', '8861892495fffff', '8860145125fffff', '8861892c3bfffff', '88618921c5fffff', '8860169621fffff', '8860169619fffff',
       '8860145aadfffff', '8860145921fffff', '88618934e3fffff', '886014c905fffff', '8860145907fffff', '88618921cdfffff', '8861892281fffff', '8860145101fffff',
         '8860145ab5fffff', '8861892117fffff', '8861893495fffff', '88601690c7fffff', '886016968dfffff', '8860145995fffff', '886189228dfffff', '886016944bfffff',
           '88618934b1fffff', '8861892f65fffff', '8860169687fffff', '8860169743fffff', '8860145115fffff', '8861892691fffff', '8860169655fffff', '8860145359fffff',
             '8861892461fffff', '8860145b3bfffff', '8861892427fffff', '8861892c4bfffff', '88618925e9fffff', '88618925c7fffff', '88618925c5fffff', '88618925c9fffff',
               '88618925c1fffff', '8861892437fffff', '8861892513fffff', '8861892c53fffff', '8861892e03fffff', '88618925edfffff', '8861892e33fffff',
                 '886189240bfffff', '8860145b43fffff', '8861892e5dfffff', '88618920adfffff', '88618925ebfffff', '8861892563fffff', '8861892cd3fffff',
                   '8861892721fffff', '8860145a59fffff', '8861892eb3fffff', '8861892e17fffff', '8861892e61fffff', '88618925e1fffff', '886189252bfffff',
                     '8861892e67fffff', '8860145b19fffff', '8860145a55fffff', '8861892603fffff', '8861892519fffff', '8861892511fffff', '8861892407fffff',
                       '8861892451fffff', '8861892e05fffff', '8861892717fffff', '8860145945fffff', '8861892ee7fffff', '8861892e45fffff', '8861892c09fffff',
                         '8861892517fffff', '88618925e3fffff', '88618926c1fffff', '8861892085fffff', '8861892625fffff', '88618920d3fffff', '886016975bfffff',
                           '8861892c69fffff', '886014534dfffff', '8860144b43fffff', '8860145acbfffff', '886014592bfffff', '8861892cb5fffff', '8861892ca9fffff',
                             '8861892da3fffff', '8860145133fffff', '8861892db9fffff', '88618926e1fffff', '8860145951fffff', '88601458c5fffff', '8860145831fffff',
                               '886014c925fffff', '8861892dc5fffff', '8861892dc7fffff', '88618926d1fffff', '88618935b7fffff', '8861892591fffff', '88618925d9fffff',
                                 '8861892425fffff', '8861892585fffff', '8861892e1bfffff', '8860145b1dfffff', '88601451adfffff', '8860145365fffff', '886014c9adfffff',
                                   '88618922b3fffff', '8860145ac9fffff', '8861892ca5fffff', '886014536bfffff', '88618935b1fffff', '8860145b37fffff',
                                     '8860145919fffff', '886014c9a9fffff', '886189269dfffff', '8860145a15fffff', '88618934b3fffff', '8860144b4dfffff',
                                       '88601451e7fffff', '8861892697fffff', '886014c923fffff', '8860145aedfffff', '886014c915fffff', '8860145341fffff',
                                         '8860145117fffff', '8861892293fffff', '8860169719fffff', '8860145363fffff', '88601459b5fffff', '8860145ae1fffff',
                                           '886014c92dfffff', '88601696e3fffff', '8861892c63fffff', '8860145113fffff', '8860144a6dfffff', '8860145315fffff',
                                             '8860144b55fffff', '886014c937fffff', '8860145abbfffff', '8860145b4bfffff', '8861892e91fffff', '8861892621fffff',
                                               '8860145b49fffff', '886189255bfffff', '8861892545fffff', '88618925d7fffff', '88618925cdfffff', '8861892e0dfffff', 
                                               '886189258dfffff', '88618925b5fffff', '88618925b7fffff', '8860145949fffff', '8861892e41fffff', '886189219bfffff',
                                                 '886189251bfffff', '886016965dfffff', '8861892539fffff', '886189265bfffff', '8861892505fffff', '88618934a3fffff',
                                                   '8861892501fffff', '8860169681fffff')
and hour in ('17', '18', '19', '20', '21')
"""

#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [16]:
#Create a copy of the df retrived from presto
df = df_copy.copy()
df = df[df['pickup_location_hex_8'].isin(hyderabad_new_zone)]
df.head()

,order_id,accept_to_pickup_distance,city_name,hhmmss,hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8
0,646376ef396d452aefced1c7,NaN,Bangalore,175831,17,Auto,17:59,2,20230516,expired,None,0.0,"[""60e97d80c07ad027ea76d06b"",""6114c8def3519d81c...",88618925c7fffff
1,6463696b11a4ec484f01a885,NaN,Bangalore,170051,17,Auto,17:00,2,20230516,customerCancelled,order cancelled before rider accepted,0.0,"[""61c284edf3de6f3fbb533403"",""5d89ed48f354c15d3...",886189258dfffff
2,64636ac19d1dd7505bb935a8,NaN,Bangalore,170633,17,Auto,17:00,2,20230516,expired,None,1.0,"[""60e554d2af791be07a8cf3a8""]",8860144b4dfffff
3,64636b724e73755d6de5efde,0.327,Bangalore,170930,17,Auto,17:00,2,20230516,dropped,None,2.0,"[""60ebf1d6c7d851367684ec14"",""63df9fb462f4ac0d1...",8860145125fffff
4,64636c934bcb4717d7727284,NaN,Bangalore,171419,17,Auto,17:00,2,20230516,customerCancelled,order cancelled before rider accepted,1.0,"[""5f756d697ea5e7c46eae0897"",""63b00d62f17659332...",886189219bfffff


In [17]:
#View the dataset
df_captain = df.copy()
df_captain = df_captain[df_captain['pickup_location_hex_8'].isin(hyderabad_new_zone)]
df_captain.head()

,order_id,accept_to_pickup_distance,city_name,hhmmss,hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8
0,646376ef396d452aefced1c7,NaN,Bangalore,175831,17,Auto,17:59,2,20230516,expired,None,0.0,"[""60e97d80c07ad027ea76d06b"",""6114c8def3519d81c...",88618925c7fffff
1,6463696b11a4ec484f01a885,NaN,Bangalore,170051,17,Auto,17:00,2,20230516,customerCancelled,order cancelled before rider accepted,0.0,"[""61c284edf3de6f3fbb533403"",""5d89ed48f354c15d3...",886189258dfffff
2,64636ac19d1dd7505bb935a8,NaN,Bangalore,170633,17,Auto,17:00,2,20230516,expired,None,1.0,"[""60e554d2af791be07a8cf3a8""]",8860144b4dfffff
3,64636b724e73755d6de5efde,0.327,Bangalore,170930,17,Auto,17:00,2,20230516,dropped,None,2.0,"[""60ebf1d6c7d851367684ec14"",""63df9fb462f4ac0d1...",8860145125fffff
4,64636c934bcb4717d7727284,NaN,Bangalore,171419,17,Auto,17:00,2,20230516,customerCancelled,order cancelled before rider accepted,1.0,"[""5f756d697ea5e7c46eae0897"",""63b00d62f17659332...",886189219bfffff


In [18]:
#Filter only on the hexes needed
df_filter_level1 = df
df_filter_level1['count'] = 1
df_filter_level1['date'] = df_filter_level1['yyyymmdd'].astype(str).apply(lambda x: x[6:])

In [19]:
#Create a new column for cancel_reason to identify OCARA
df_filter_level1['new_cancel_reason'] = np.where(df_filter_level1['order_status'] == 'customerCancelled',
 np.where(df_filter_level1['cancel_reason'] == 'order cancelled before rider accepted', 'COBRA',
 np.where(df_filter_level1['cancel_reason'] == 'Order cancelled before rider was mapped', 'COBRM', 'OCARA')), 'no_cancellation')

In [20]:
#Check if the order is expiry_mapped or expiry_unmapped
df_filter_level1['expiry_type'] = np.where(df_filter_level1['order_status'] == 'expired',
 np.where(df_filter_level1['map_riders'] == '[]', 'expiry_unmapped', 'expiry_mapped'), 'no_expiry')
df_filter_level1['final_order_status'] = np.where(df_filter_level1['order_status'] == 'customerCancelled', df_filter_level1['new_cancel_reason'],
np.where(df_filter_level1['order_status'] == 'expired', df_filter_level1['expiry_type'], df_filter_level1['order_status']))

In [21]:
#Add segment and label to the datasets
df_filter_level1 = df_filter_level1.merge(hexes, on = 'pickup_location_hex_8', how = 'left')
df_captain = df_captain.merge(hexes, on = 'pickup_location_hex_8', how = 'left')
df_filter_level1.head()

,order_id,accept_to_pickup_distance,city_name,hhmmss,hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8,count,date,new_cancel_reason,expiry_type,final_order_status,Segment,Label
0,646376ef396d452aefced1c7,NaN,Bangalore,175831,17,Auto,17:59,2,20230516,expired,None,0.0,"[""60e97d80c07ad027ea76d06b"",""6114c8def3519d81c...",88618925c7fffff,1,16,no_cancellation,expiry_mapped,expiry_mapped,in-high,Zone
1,6463696b11a4ec484f01a885,NaN,Bangalore,170051,17,Auto,17:00,2,20230516,customerCancelled,order cancelled before rider accepted,0.0,"[""61c284edf3de6f3fbb533403"",""5d89ed48f354c15d3...",886189258dfffff,1,16,COBRA,no_expiry,COBRA,out-high,Holdout
2,64636ac19d1dd7505bb935a8,NaN,Bangalore,170633,17,Auto,17:00,2,20230516,expired,None,1.0,"[""60e554d2af791be07a8cf3a8""]",8860144b4dfffff,1,16,no_cancellation,expiry_mapped,expiry_mapped,out-low,Holdout
3,64636b724e73755d6de5efde,0.327,Bangalore,170930,17,Auto,17:00,2,20230516,dropped,None,2.0,"[""60ebf1d6c7d851367684ec14"",""63df9fb462f4ac0d1...",8860145125fffff,1,16,no_cancellation,no_expiry,dropped,in-low,Holdout
4,64636c934bcb4717d7727284,NaN,Bangalore,171419,17,Auto,17:00,2,20230516,customerCancelled,order cancelled before rider accepted,1.0,"[""5f756d697ea5e7c46eae0897"",""63b00d62f17659332...",886189219bfffff,1,16,COBRA,no_expiry,COBRA,out-high,Holdout


In [22]:
#Metrics_pivot
metrics_pivot = pd.pivot_table(df_filter_level1, index = ['yyyymmdd', 'hour', 'Segment', 'Label'], columns = 'final_order_status',
 values = 'count', aggfunc = 'sum', fill_value=0).reset_index().rename_axis(None, axis=1)
metrics_pivot['total'] = metrics_pivot[['COBRA', 'COBRM', 'OCARA', 'aborted', 'dropped', 'expiry_mapped', 'expiry_unmapped']].sum(axis = 1)
metrics_pivot['OCARA%'] = round(metrics_pivot['OCARA']/metrics_pivot['total'], 4)
metrics_pivot['COBRA%'] = round(metrics_pivot['COBRA']/metrics_pivot['total'], 4)
metrics_pivot['COBRM%'] = round(metrics_pivot['COBRM']/metrics_pivot['total'], 4)
metrics_pivot['G2N'] = round(metrics_pivot['dropped']/metrics_pivot['total'], 4)
metrics_pivot['expiry_mapped%'] = round(metrics_pivot['expiry_mapped']/metrics_pivot['total'], 4)
if 'expiry_unmapped' in metrics_pivot.columns:
    metrics_pivot['Stockout%'] = round(metrics_pivot['expiry_unmapped']/metrics_pivot['total'], 4)
else:
    metrics_pivot['Stockout%'] = 0
metrics_df = metrics_pivot[['yyyymmdd', 'hour', 'Segment', 'Label', 'OCARA%', 'COBRA%', 'COBRM%', 'G2N', 'expiry_mapped%', 'Stockout%', 'total']]
metrics_df.head()

,yyyymmdd,hour,Segment,Label,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total
0,20230424,17,balanced-high,Holdout,0.0759,0.4107,0.0,0.3259,0.1696,0.0179,224
1,20230424,17,balanced-high,Zone,0.0933,0.2800,0.0,0.3200,0.3067,0.0000,75
2,20230424,17,balanced-low,Holdout,0.1373,0.2157,0.0,0.2941,0.3333,0.0196,51
3,20230424,17,in-high,Holdout,0.0801,0.3406,0.0,0.3304,0.2460,0.0029,687
4,20230424,17,in-high,Zone,0.0607,0.3457,0.0,0.2243,0.3692,0.0000,1021


In [23]:
#Get the average ETA and FM
fm_eta = df_filter_level1.groupby(['yyyymmdd', 'hour', 'Segment', 'Label'])[['accept_to_pickup_distance', 'eta']].median().reset_index()
fm_eta = fm_eta.rename(columns = {'accept_to_pickup_distance':'FM'})
metrics_df = metrics_df.merge(fm_eta, on = ['yyyymmdd', 'hour', 'Segment', 'Label'], how = 'left')
metrics_df.head()

,yyyymmdd,hour,Segment,Label,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total,FM,eta
0,20230424,17,balanced-high,Holdout,0.0759,0.4107,0.0,0.3259,0.1696,0.0179,224,0.8490,3.0
1,20230424,17,balanced-high,Zone,0.0933,0.2800,0.0,0.3200,0.3067,0.0000,75,0.5195,2.0
2,20230424,17,balanced-low,Holdout,0.1373,0.2157,0.0,0.2941,0.3333,0.0196,51,0.8910,3.0
3,20230424,17,in-high,Holdout,0.0801,0.3406,0.0,0.3304,0.2460,0.0029,687,0.8260,3.0
4,20230424,17,in-high,Zone,0.0607,0.3457,0.0,0.2243,0.3692,0.0000,1021,0.7660,3.0


In [24]:
#Get the number of captains by cluster
#Make columns for second and minute and date
df_captain['second'] = df_captain['hhmmss'].astype(str).apply(lambda x: x[4:])
df_captain['minute'] = df_captain['hhmmss'].astype(str).apply(lambda x: x[2:4])
df_captain['date'] = df_captain['yyyymmdd'].astype(str).apply(lambda x: x[6:])
df_captain['map_riders'] = df_captain['map_riders'].apply(lambda x: json.loads(x))
df_captain = df_captain[df_captain['map_riders'].apply(lambda x: len(x)) > 0]
df_captain['15_sec_batch'] = pd.cut(df_captain['second'].astype(int), bins = [-1, 15, 30, 45, 61], labels = ['0-15', '15-30', '30-45', '45-60'])

#Get the riders mapped in 20_sec_windows
df_captains_20_sec = df_captain.groupby(['yyyymmdd', 'hour', 'Segment', 'Label', 'minute', '15_sec_batch'])['map_riders'].apply(list).reset_index()

#Get the number of captains in a single list
map_riders = []
df_captains_20_sec = df_captains_20_sec.dropna()
for x in df_captains_20_sec['map_riders'].values:
    riders = [item for sublist in x for item in sublist]
    map_riders.append(riders)

unique_captains = []
for x in map_riders:
    unique_captains_list = []
    for y in x:
        if y in unique_captains_list:
            continue
        else:
            unique_captains_list.append(y)
    unique_captains.append(unique_captains_list)

df_captains_20_sec['total_captains'] = unique_captains

#Get the number of captains in each batch
df_captains_20_sec['number_of_captains'] = df_captains_20_sec['total_captains'].apply(lambda x: len(x))

#Get the average number of captains coming in each 20_second_window
df_20_sec_average = df_captains_20_sec.groupby(['yyyymmdd', 'hour', 'Segment', 'Label'])['number_of_captains'].median().reset_index()

#Merge the hourly captains and 20_sec_captains with the metrics table
metrics_df = metrics_df.merge(df_20_sec_average, on = ['yyyymmdd', 'hour', 'Segment', 'Label'], how = 'left')
metrics_df.head()

,yyyymmdd,hour,Segment,Label,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total,FM,eta,number_of_captains
0,20230424,17,balanced-high,Holdout,0.0759,0.4107,0.0,0.3259,0.1696,0.0179,224,0.8490,3.0,36.5
1,20230424,17,balanced-high,Zone,0.0933,0.2800,0.0,0.3200,0.3067,0.0000,75,0.5195,2.0,52.0
2,20230424,17,balanced-low,Holdout,0.1373,0.2157,0.0,0.2941,0.3333,0.0196,51,0.8910,3.0,8.0
3,20230424,17,in-high,Holdout,0.0801,0.3406,0.0,0.3304,0.2460,0.0029,687,0.8260,3.0,55.5
4,20230424,17,in-high,Zone,0.0607,0.3457,0.0,0.2243,0.3692,0.0000,1021,0.7660,3.0,65.0


In [29]:
np.argsort(metrics_df['total'].values)[::-1]

array([202, 157, 247, 103, 193, 112,  22, 148, 211, 238, 204, 201,  58,
        67, 195, 159, 150, 240, 249,  13, 256, 105, 166,  60,  94, 114,
        69, 156, 121, 246,  15,  31, 139, 111, 184,  24, 213, 141, 192,
       229, 102, 237, 231, 147, 186, 255, 258,  96, 168, 210,  49,  76,
       220,  21,   4, 265,  66,  57, 175, 165, 123,   6,  12,  51, 198,
        33,  40, 120, 130, 228, 138,  93, 205,  30, 183,  78, 153, 267,
       264,  75,   3, 219,  85,  48, 177, 108, 189, 222, 132,  99, 234,
       144,  42, 174, 243, 129,  39,  18, 106,  87,  63,  84, 196, 250,
       160, 115,  25, 207,   9,  54, 241, 199, 214, 252, 180,  16, 135,
       151, 162, 225, 171,  27,  70,  61, 154, 261, 117, 216,  45, 167,
       113,  90, 158, 140, 208, 194,  72, 259, 248, 239,  36, 244, 203,
         0,  34, 149, 126,  97, 104, 176, 124, 230, 257, 169, 187, 212,
       185,  64,  23, 190, 109, 142,  14, 235, 163, 221, 122,  10,  68,
       145,  95,  32,  81,  52,   5, 266,  55, 253,  50,  77,  8

In [32]:
metrics_df['total'].max()

4768

In [31]:
metrics_df.loc[202]

yyyymmdd              20230516
hour                        19
Segment                in-high
Label                     Zone
OCARA%                  0.0151
COBRA%                  0.2838
COBRM%                  0.0002
G2N                     0.0304
expiry_mapped%          0.6705
Stockout%                  0.0
total                     4768
FM                       0.813
eta                        0.0
number_of_captains        66.0
Name: 202, dtype: object

In [25]:
#Query data from order_logs_immutable for calculating TTC

q = """
select order_id, accept_to_pickup_distance, captain_location_hex_8, captain_location_latitude,
captain_location_longitude, city_name, customer_location_hex_8, drop_location_hex_8, drop_location_latitude, drop_location_longitude, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, updated_epoch, event_type, pickup_location_hex_8
from orders.order_logs_immutable
where yyyymmdd in ('20230424', '20230425', '20230426', '20230515', '20230516', '20230517')
and city_name in ('Bangalore')
and service_obj_service_name = 'Auto'
and event_type in ('customer_cancelled', 'order_requested', 'accepted')
and pickup_location_hex_8 in ('88618924bbfffff', '886189260bfffff', '8860145913fffff', '88618924b3fffff', '8860145823fffff', '8860169659fffff', '886014595dfffff',
 '8860145a07fffff', '8860145847fffff', '8860169607fffff', '8861892c81fffff', '88601451e1fffff', '8861892687fffff', '88618921e3fffff', '8861892187fffff',
   '88601451e5fffff', '8860145acdfffff', '88618920ebfffff', '886189203dfffff', '8860145a83fffff', '8860145a87fffff', '886189208dfffff', '8860169645fffff',
     '88601696dbfffff', '8861892debfffff', '8861892495fffff', '8860145125fffff', '8861892c3bfffff', '88618921c5fffff', '8860169621fffff', '8860169619fffff',
       '8860145aadfffff', '8860145921fffff', '88618934e3fffff', '886014c905fffff', '8860145907fffff', '88618921cdfffff', '8861892281fffff', '8860145101fffff',
         '8860145ab5fffff', '8861892117fffff', '8861893495fffff', '88601690c7fffff', '886016968dfffff', '8860145995fffff', '886189228dfffff', '886016944bfffff',
           '88618934b1fffff', '8861892f65fffff', '8860169687fffff', '8860169743fffff', '8860145115fffff', '8861892691fffff', '8860169655fffff', '8860145359fffff',
             '8861892461fffff', '8860145b3bfffff', '8861892427fffff', '8861892c4bfffff', '88618925e9fffff', '88618925c7fffff', '88618925c5fffff', '88618925c9fffff',
               '88618925c1fffff', '8861892437fffff', '8861892513fffff', '8861892c53fffff', '8861892e03fffff', '88618925edfffff', '8861892e33fffff',
                 '886189240bfffff', '8860145b43fffff', '8861892e5dfffff', '88618920adfffff', '88618925ebfffff', '8861892563fffff', '8861892cd3fffff',
                   '8861892721fffff', '8860145a59fffff', '8861892eb3fffff', '8861892e17fffff', '8861892e61fffff', '88618925e1fffff', '886189252bfffff',
                     '8861892e67fffff', '8860145b19fffff', '8860145a55fffff', '8861892603fffff', '8861892519fffff', '8861892511fffff', '8861892407fffff',
                       '8861892451fffff', '8861892e05fffff', '8861892717fffff', '8860145945fffff', '8861892ee7fffff', '8861892e45fffff', '8861892c09fffff',
                         '8861892517fffff', '88618925e3fffff', '88618926c1fffff', '8861892085fffff', '8861892625fffff', '88618920d3fffff', '886016975bfffff',
                           '8861892c69fffff', '886014534dfffff', '8860144b43fffff', '8860145acbfffff', '886014592bfffff', '8861892cb5fffff', '8861892ca9fffff',
                             '8861892da3fffff', '8860145133fffff', '8861892db9fffff', '88618926e1fffff', '8860145951fffff', '88601458c5fffff', '8860145831fffff',
                               '886014c925fffff', '8861892dc5fffff', '8861892dc7fffff', '88618926d1fffff', '88618935b7fffff', '8861892591fffff', '88618925d9fffff',
                                 '8861892425fffff', '8861892585fffff', '8861892e1bfffff', '8860145b1dfffff', '88601451adfffff', '8860145365fffff', '886014c9adfffff',
                                   '88618922b3fffff', '8860145ac9fffff', '8861892ca5fffff', '886014536bfffff', '88618935b1fffff', '8860145b37fffff',
                                     '8860145919fffff', '886014c9a9fffff', '886189269dfffff', '8860145a15fffff', '88618934b3fffff', '8860144b4dfffff',
                                       '88601451e7fffff', '8861892697fffff', '886014c923fffff', '8860145aedfffff', '886014c915fffff', '8860145341fffff',
                                         '8860145117fffff', '8861892293fffff', '8860169719fffff', '8860145363fffff', '88601459b5fffff', '8860145ae1fffff',
                                           '886014c92dfffff', '88601696e3fffff', '8861892c63fffff', '8860145113fffff', '8860144a6dfffff', '8860145315fffff',
                                             '8860144b55fffff', '886014c937fffff', '8860145abbfffff', '8860145b4bfffff', '8861892e91fffff', '8861892621fffff',
                                               '8860145b49fffff', '886189255bfffff', '8861892545fffff', '88618925d7fffff', '88618925cdfffff', '8861892e0dfffff', 
                                               '886189258dfffff', '88618925b5fffff', '88618925b7fffff', '8860145949fffff', '8861892e41fffff', '886189219bfffff',
                                                 '886189251bfffff', '886016965dfffff', '8861892539fffff', '886189265bfffff', '8861892505fffff', '88618934a3fffff',
                                                   '8861892501fffff', '8860169681fffff')
and hour in ('17', '18', '19', '20', '21')
"""

#Load data into pandas table
df_ttc_copy = pd.read_sql(q, connection)

KeyboardInterrupt: 

In [ ]:
#Load and view dataset
df_ttc = df_ttc_copy.copy()
df_ttc = df_ttc[df_ttc['pickup_location_hex_8'].isin(hyderabad_new_zone)]
df_ttc['timestamp'] = df_ttc['updated_epoch'].apply(lambda x: datetime.fromtimestamp(x/1000))
df_ttc = df_ttc.sort_values(by = 'timestamp')
df_ttc.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,pickup_location_hex_8,timestamp
98581,644273bcbccb1c20d1edc4fe,NaN,,NaN,NaN,Bangalore,88618925ebfffff,8861892589fffff,12.931780,77.608199,12.937325,77.619543,170004,17,1700,Auto,17:00,5,20230421,requested,None,1682076604160,order_requested,88618925ebfffff,2023-04-21 17:00:04.160
136645,644273be9e36a256b0248e09,NaN,,NaN,NaN,Bangalore,88618925e1fffff,88618925a7fffff,12.973612,77.606870,12.947659,77.625725,170006,17,1700,Auto,17:00,5,20230421,requested,None,1682076606776,order_requested,88618925e1fffff,2023-04-21 17:00:06.776
146782,644273bfcbd2b258d3796349,NaN,,NaN,NaN,Bangalore,886189255bfffff,886189240dfffff,12.884806,77.640769,12.908676,77.651927,170007,17,1700,Auto,17:00,5,20230421,requested,None,1682076607046,order_requested,886189255bfffff,2023-04-21 17:00:07.046
150720,644273c0d1a99767ebcfe158,NaN,,NaN,NaN,Bangalore,88618925e9fffff,88618925c9fffff,12.916585,77.621930,12.936978,77.626800,170008,17,1700,Auto,17:00,5,20230421,requested,None,1682076608795,order_requested,88618925e9fffff,2023-04-21 17:00:08.795
128375,644273c12937c94eff828917,NaN,,NaN,NaN,Bangalore,88618925d9fffff,88618924a7fffff,12.921753,77.574513,12.911766,77.597809,170009,17,1700,Auto,17:00,5,20230421,requested,None,1682076609080,order_requested,88618925d9fffff,2023-04-21 17:00:09.080


In [ ]:
#Create a column to identify OCARA, COBRA and COBRM
df_ttc['new_cancel_reason'] = np.where(df_ttc['event_type'] == 'customer_cancelled',
 np.where(df_ttc['cancel_reason'] == 'order cancelled before rider accepted', 'COBRA',
 np.where(df_ttc['cancel_reason'] == 'Order cancelled before rider was mapped', 'COBRM', 'OCARA')), 'no_cancellation')
df_ttc.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,pickup_location_hex_8,timestamp,new_cancel_reason
98581,644273bcbccb1c20d1edc4fe,NaN,,NaN,NaN,Bangalore,88618925ebfffff,8861892589fffff,12.931780,77.608199,12.937325,77.619543,170004,17,1700,Auto,17:00,5,20230421,requested,None,1682076604160,order_requested,88618925ebfffff,2023-04-21 17:00:04.160,no_cancellation
136645,644273be9e36a256b0248e09,NaN,,NaN,NaN,Bangalore,88618925e1fffff,88618925a7fffff,12.973612,77.606870,12.947659,77.625725,170006,17,1700,Auto,17:00,5,20230421,requested,None,1682076606776,order_requested,88618925e1fffff,2023-04-21 17:00:06.776,no_cancellation
146782,644273bfcbd2b258d3796349,NaN,,NaN,NaN,Bangalore,886189255bfffff,886189240dfffff,12.884806,77.640769,12.908676,77.651927,170007,17,1700,Auto,17:00,5,20230421,requested,None,1682076607046,order_requested,886189255bfffff,2023-04-21 17:00:07.046,no_cancellation
150720,644273c0d1a99767ebcfe158,NaN,,NaN,NaN,Bangalore,88618925e9fffff,88618925c9fffff,12.916585,77.621930,12.936978,77.626800,170008,17,1700,Auto,17:00,5,20230421,requested,None,1682076608795,order_requested,88618925e9fffff,2023-04-21 17:00:08.795,no_cancellation
128375,644273c12937c94eff828917,NaN,,NaN,NaN,Bangalore,88618925d9fffff,88618924a7fffff,12.921753,77.574513,12.911766,77.597809,170009,17,1700,Auto,17:00,5,20230421,requested,None,1682076609080,order_requested,88618925d9fffff,2023-04-21 17:00:09.080,no_cancellation


In [ ]:
#Filter the df only zone1
df_ttc_filter = df_ttc
df_ttc_filter = df_ttc_filter.merge(hexes, on = 'pickup_location_hex_8', how = 'left')
df_ttc_filter.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,pickup_location_hex_8,timestamp,new_cancel_reason,Segment,Label
0,644273bcbccb1c20d1edc4fe,NaN,,NaN,NaN,Bangalore,88618925ebfffff,8861892589fffff,12.931780,77.608199,12.937325,77.619543,170004,17,1700,Auto,17:00,5,20230421,requested,None,1682076604160,order_requested,88618925ebfffff,2023-04-21 17:00:04.160,no_cancellation,in-high,Zone
1,644273be9e36a256b0248e09,NaN,,NaN,NaN,Bangalore,88618925e1fffff,88618925a7fffff,12.973612,77.606870,12.947659,77.625725,170006,17,1700,Auto,17:00,5,20230421,requested,None,1682076606776,order_requested,88618925e1fffff,2023-04-21 17:00:06.776,no_cancellation,in-high,Zone
2,644273bfcbd2b258d3796349,NaN,,NaN,NaN,Bangalore,886189255bfffff,886189240dfffff,12.884806,77.640769,12.908676,77.651927,170007,17,1700,Auto,17:00,5,20230421,requested,None,1682076607046,order_requested,886189255bfffff,2023-04-21 17:00:07.046,no_cancellation,out-high,Holdout
3,644273c0d1a99767ebcfe158,NaN,,NaN,NaN,Bangalore,88618925e9fffff,88618925c9fffff,12.916585,77.621930,12.936978,77.626800,170008,17,1700,Auto,17:00,5,20230421,requested,None,1682076608795,order_requested,88618925e9fffff,2023-04-21 17:00:08.795,no_cancellation,in-high,Holdout
4,644273c12937c94eff828917,NaN,,NaN,NaN,Bangalore,88618925d9fffff,88618924a7fffff,12.921753,77.574513,12.911766,77.597809,170009,17,1700,Auto,17:00,5,20230421,requested,None,1682076609080,order_requested,88618925d9fffff,2023-04-21 17:00:09.080,no_cancellation,balanced-high,Holdout


In [ ]:
#Filter df on the columns needed
df_filter = df_ttc_filter[['order_id', 'pickup_location_hex_8', 'hour', 'yyyymmdd', 'timestamp', 'event_type', 'new_cancel_reason', 'Segment', 'Label']]

#Filter on the respective order_status
df_cobra = df_filter[df_filter['new_cancel_reason'] == 'COBRA']
df_ocara = df_filter[df_filter['new_cancel_reason'] == 'OCARA']
df_cobrm = df_filter[df_filter['new_cancel_reason'] == 'COBRM']
df_order_created = df_filter[df_filter['event_type'] == 'order_requested']
df_accepted = df_filter[df_filter['event_type'] == 'accepted']

#Sort the dfs by timestamp
df_cobra = df_cobra.sort_values(by = 'timestamp', ascending=True)
df_cobra = df_cobra.drop_duplicates(subset='order_id')

df_ocara = df_ocara.sort_values(by = 'timestamp', ascending=True)
df_ocara = df_ocara.drop_duplicates(subset='order_id')

df_cobrm = df_cobrm.sort_values(by = 'timestamp', ascending=True)
df_cobrm = df_cobrm.drop_duplicates(subset='order_id')

df_accepted = df_accepted.sort_values(by = 'timestamp', ascending=True)
df_accepted = df_accepted.drop_duplicates(subset='order_id')

#Change the column names
df_cobra = df_cobra[['order_id', 'new_cancel_reason', 'timestamp']]
df_cobra.columns = ['order_id', 'COBRA', 'timestamp_cobra']

df_cobrm = df_cobrm[['order_id', 'new_cancel_reason', 'timestamp']]
df_cobrm.columns = ['order_id', 'COBRM', 'timestamp_cobrm']

df_ocara = df_ocara[['order_id', 'new_cancel_reason', 'timestamp']]
df_ocara.columns = ['order_id', 'OCARA', 'timestamp_ocara']

df_accepted = df_accepted[['order_id', 'event_type', 'timestamp']]
df_accepted.columns = ['order_id', 'Accepted', 'timestamp_accepted']

#Merge the dfs together
dfs_to_merge = [df_order_created, df_cobra, df_cobrm, df_ocara, df_accepted]
df_final = reduce(lambda  left, right: pd.merge(left, right,on=['order_id'], how='left'), dfs_to_merge).fillna('NA')
df_final.head()

,order_id,pickup_location_hex_8,hour,yyyymmdd,timestamp,event_type,new_cancel_reason,Segment,Label,COBRA,timestamp_cobra,COBRM,timestamp_cobrm,OCARA,timestamp_ocara,Accepted,timestamp_accepted
0,644273bcbccb1c20d1edc4fe,88618925ebfffff,17,20230421,2023-04-21 17:00:04.160,order_requested,no_cancellation,in-high,Zone,NA,NA,NA,NA,NA,NA,NA,NA
1,644273be9e36a256b0248e09,88618925e1fffff,17,20230421,2023-04-21 17:00:06.776,order_requested,no_cancellation,in-high,Zone,NA,NA,NA,NA,NA,NA,NA,NA
2,644273bfcbd2b258d3796349,886189255bfffff,17,20230421,2023-04-21 17:00:07.046,order_requested,no_cancellation,out-high,Holdout,NA,NA,NA,NA,NA,NA,accepted,2023-04-21 17:01:23.222000
3,644273c0d1a99767ebcfe158,88618925e9fffff,17,20230421,2023-04-21 17:00:08.795,order_requested,no_cancellation,in-high,Holdout,NA,NA,NA,NA,NA,NA,accepted,2023-04-21 17:01:21.978000
4,644273c12937c94eff828917,88618925d9fffff,17,20230421,2023-04-21 17:00:09.080,order_requested,no_cancellation,balanced-high,Holdout,COBRA,2023-04-21 17:00:43.124000,NA,NA,NA,NA,NA,NA


In [ ]:
#Filter on the respective order_status
df_cobra = df_final[df_final['COBRA'] == 'COBRA']
df_ocara = df_final[df_final['OCARA'] == 'OCARA']
df_cobrm = df_final[df_final['COBRM'] == 'COBRM']
df_accepted = df_final[df_final['Accepted'] == 'accepted']

#Calculate TTC for cobrm, ocara, and cobra and tta
df_cobra['ttc_cobra'] = pd.to_datetime(df_cobra['timestamp_cobra']) - pd.to_datetime(df_cobra['timestamp'])
df_cobra['ttc_cobra'] = df_cobra['ttc_cobra'].apply(lambda x: x.total_seconds())

df_cobrm['ttc_cobrm'] = pd.to_datetime(df_cobrm['timestamp_cobrm']) - pd.to_datetime(df_cobrm['timestamp'])
df_cobrm['ttc_cobrm'] = df_cobrm['ttc_cobrm'].apply(lambda x: x.total_seconds())

df_ocara['ttc_ocara'] = pd.to_datetime(df_ocara['timestamp_ocara']) - pd.to_datetime(df_ocara['timestamp'])
df_ocara['ttc_ocara'] = df_ocara['ttc_ocara'].apply(lambda x: x.total_seconds())

df_accepted['tta'] = pd.to_datetime(df_accepted['timestamp_accepted']) - pd.to_datetime(df_accepted['timestamp'])
df_accepted['tta'] = df_accepted['tta'].apply(lambda x: x.total_seconds())

#Filter only on the new columns made
df_cobra = df_cobra[['order_id', 'ttc_cobra']]
df_cobrm = df_cobrm[['order_id', 'ttc_cobrm']]
df_ocara = df_ocara[['order_id', 'ttc_ocara']]
df_accepted = df_accepted[['order_id', 'tta']]

#Merge the dfs together
dfs_to_merge = [df_final, df_cobra, df_cobrm, df_ocara, df_accepted]
df_final = reduce(lambda  left, right: pd.merge(left, right,on=['order_id'], how='left'), dfs_to_merge)
df_final.head()

,order_id,pickup_location_hex_8,hour,yyyymmdd,timestamp,event_type,new_cancel_reason,Segment,Label,COBRA,timestamp_cobra,COBRM,timestamp_cobrm,OCARA,timestamp_ocara,Accepted,timestamp_accepted,ttc_cobra,ttc_cobrm,ttc_ocara,tta
0,644273bcbccb1c20d1edc4fe,88618925ebfffff,17,20230421,2023-04-21 17:00:04.160,order_requested,no_cancellation,in-high,Zone,NA,NA,NA,NA,NA,NA,NA,NA,NaN,NaN,NaN,NaN
1,644273be9e36a256b0248e09,88618925e1fffff,17,20230421,2023-04-21 17:00:06.776,order_requested,no_cancellation,in-high,Zone,NA,NA,NA,NA,NA,NA,NA,NA,NaN,NaN,NaN,NaN
2,644273bfcbd2b258d3796349,886189255bfffff,17,20230421,2023-04-21 17:00:07.046,order_requested,no_cancellation,out-high,Holdout,NA,NA,NA,NA,NA,NA,accepted,2023-04-21 17:01:23.222000,NaN,NaN,NaN,76.176
3,644273c0d1a99767ebcfe158,88618925e9fffff,17,20230421,2023-04-21 17:00:08.795,order_requested,no_cancellation,in-high,Holdout,NA,NA,NA,NA,NA,NA,accepted,2023-04-21 17:01:21.978000,NaN,NaN,NaN,73.183
4,644273c12937c94eff828917,88618925d9fffff,17,20230421,2023-04-21 17:00:09.080,order_requested,no_cancellation,balanced-high,Holdout,COBRA,2023-04-21 17:00:43.124000,NA,NA,NA,NA,NA,NA,34.044,NaN,NaN,NaN


In [ ]:
#Create a date column
df_final['date'] = df_final['yyyymmdd'].astype(str).apply(lambda x: x[6:])
df_final['count'] = 1

#Get average ttc, tta
df_cobra_grouped = df_final[df_final['COBRA'] == 'COBRA'].groupby(['yyyymmdd', 'hour', 'Segment', 'Label'])['ttc_cobra'].median().reset_index()
df_ocara_grouped = df_final[df_final['OCARA'] == 'OCARA'].groupby(['yyyymmdd', 'hour', 'Segment', 'Label'])['ttc_ocara'].median().reset_index()
df_cobrm_grouped = df_final[df_final['COBRM'] == 'COBRM'].groupby(['yyyymmdd', 'hour', 'Segment', 'Label'])['ttc_cobrm'].median().reset_index()
df_accepted_grouped = df_final[df_final['Accepted'] == 'accepted'].groupby(['yyyymmdd', 'hour', 'Segment', 'Label'])['tta'].median().reset_index()

#Merge the dfs together
dfs_to_merge = [df_cobra_grouped, df_cobrm_grouped, df_ocara_grouped, df_accepted_grouped]
df_metrics_final = reduce(lambda  left, right: pd.merge(left, right,on=['yyyymmdd', 'hour', 'Segment', 'Label'], how='left'), dfs_to_merge).fillna('NA')
df_metrics_final.head()

,yyyymmdd,hour,Segment,Label,ttc_cobra,ttc_cobrm,ttc_ocara,tta
0,20230421,17,balanced-high,Holdout,89.715,NA,347.593,11.460
1,20230421,17,balanced-high,Zone,102.802,NA,157.3835,37.074
2,20230421,17,balanced-low,Holdout,120.720,NA,148.46,9.057
3,20230421,17,in-high,Holdout,90.416,32.572,238.538,11.828
4,20230421,17,in-high,Zone,107.732,NA,222.206,17.916


In [ ]:
#Merge ttc with metrics_df
metrics_df = metrics_df.merge(df_metrics_final, on = ['yyyymmdd', 'hour', 'Segment', 'Label'], how = 'left')
metrics_df.head()

,yyyymmdd,hour,Segment,Label,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total,FM,eta,number_of_captains,ttc_cobra,ttc_cobrm,ttc_ocara,tta
0,20230421,17,balanced-high,Holdout,0.0969,0.2984,0.0000,0.4419,0.1628,0.0000,258,13.423963,2.899225,49.0,89.715,NA,347.593,11.460
1,20230421,17,balanced-high,Zone,0.0606,0.4545,0.0000,0.1919,0.2929,0.0000,99,64.619934,3.454545,33.0,102.802,NA,157.3835,37.074
2,20230421,17,balanced-low,Holdout,0.2113,0.3239,0.0000,0.2958,0.1690,0.0000,71,1.356786,3.647887,8.0,120.720,NA,148.46,9.057
3,20230421,17,in-high,Holdout,0.1170,0.2591,0.0042,0.3412,0.2772,0.0014,718,0.890608,3.004172,63.5,90.416,32.572,238.538,11.828
4,20230421,17,in-high,Zone,0.0882,0.3594,0.0000,0.2601,0.2914,0.0000,1088,27.258625,2.719523,81.0,107.732,NA,222.206,17.916


In [ ]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, captain_location_hex_8, captain_location_latitude,
captain_location_longitude, city_name, customer_location_hex_8, drop_location_hex_8, drop_location_latitude, drop_location_longitude, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, updated_epoch, event_type, captain_id, spd_fraud_flag, pickup_location_hex_8
from orders.order_logs_immutable
where yyyymmdd in ('20230424', '20230425', '20230426', '20230515', '20230516', '20230517')
and service_obj_service_name = 'Auto'
and event_type in ('accepted', 'rider_busy', 'rider_reject', 'dropped')
and pickup_location_hex_8 in ('88618924bbfffff', '886189260bfffff', '8860145913fffff', '88618924b3fffff', '8860145823fffff', '8860169659fffff', '886014595dfffff',
 '8860145a07fffff', '8860145847fffff', '8860169607fffff', '8861892c81fffff', '88601451e1fffff', '8861892687fffff', '88618921e3fffff', '8861892187fffff',
   '88601451e5fffff', '8860145acdfffff', '88618920ebfffff', '886189203dfffff', '8860145a83fffff', '8860145a87fffff', '886189208dfffff', '8860169645fffff',
     '88601696dbfffff', '8861892debfffff', '8861892495fffff', '8860145125fffff', '8861892c3bfffff', '88618921c5fffff', '8860169621fffff', '8860169619fffff',
       '8860145aadfffff', '8860145921fffff', '88618934e3fffff', '886014c905fffff', '8860145907fffff', '88618921cdfffff', '8861892281fffff', '8860145101fffff',
         '8860145ab5fffff', '8861892117fffff', '8861893495fffff', '88601690c7fffff', '886016968dfffff', '8860145995fffff', '886189228dfffff', '886016944bfffff',
           '88618934b1fffff', '8861892f65fffff', '8860169687fffff', '8860169743fffff', '8860145115fffff', '8861892691fffff', '8860169655fffff', '8860145359fffff',
             '8861892461fffff', '8860145b3bfffff', '8861892427fffff', '8861892c4bfffff', '88618925e9fffff', '88618925c7fffff', '88618925c5fffff', '88618925c9fffff',
               '88618925c1fffff', '8861892437fffff', '8861892513fffff', '8861892c53fffff', '8861892e03fffff', '88618925edfffff', '8861892e33fffff',
                 '886189240bfffff', '8860145b43fffff', '8861892e5dfffff', '88618920adfffff', '88618925ebfffff', '8861892563fffff', '8861892cd3fffff',
                   '8861892721fffff', '8860145a59fffff', '8861892eb3fffff', '8861892e17fffff', '8861892e61fffff', '88618925e1fffff', '886189252bfffff',
                     '8861892e67fffff', '8860145b19fffff', '8860145a55fffff', '8861892603fffff', '8861892519fffff', '8861892511fffff', '8861892407fffff',
                       '8861892451fffff', '8861892e05fffff', '8861892717fffff', '8860145945fffff', '8861892ee7fffff', '8861892e45fffff', '8861892c09fffff',
                         '8861892517fffff', '88618925e3fffff', '88618926c1fffff', '8861892085fffff', '8861892625fffff', '88618920d3fffff', '886016975bfffff',
                           '8861892c69fffff', '886014534dfffff', '8860144b43fffff', '8860145acbfffff', '886014592bfffff', '8861892cb5fffff', '8861892ca9fffff',
                             '8861892da3fffff', '8860145133fffff', '8861892db9fffff', '88618926e1fffff', '8860145951fffff', '88601458c5fffff', '8860145831fffff',
                               '886014c925fffff', '8861892dc5fffff', '8861892dc7fffff', '88618926d1fffff', '88618935b7fffff', '8861892591fffff', '88618925d9fffff',
                                 '8861892425fffff', '8861892585fffff', '8861892e1bfffff', '8860145b1dfffff', '88601451adfffff', '8860145365fffff', '886014c9adfffff',
                                   '88618922b3fffff', '8860145ac9fffff', '8861892ca5fffff', '886014536bfffff', '88618935b1fffff', '8860145b37fffff',
                                     '8860145919fffff', '886014c9a9fffff', '886189269dfffff', '8860145a15fffff', '88618934b3fffff', '8860144b4dfffff',
                                       '88601451e7fffff', '8861892697fffff', '886014c923fffff', '8860145aedfffff', '886014c915fffff', '8860145341fffff',
                                         '8860145117fffff', '8861892293fffff', '8860169719fffff', '8860145363fffff', '88601459b5fffff', '8860145ae1fffff',
                                           '886014c92dfffff', '88601696e3fffff', '8861892c63fffff', '8860145113fffff', '8860144a6dfffff', '8860145315fffff',
                                             '8860144b55fffff', '886014c937fffff', '8860145abbfffff', '8860145b4bfffff', '8861892e91fffff', '8861892621fffff',
                                               '8860145b49fffff', '886189255bfffff', '8861892545fffff', '88618925d7fffff', '88618925cdfffff', '8861892e0dfffff', 
                                               '886189258dfffff', '88618925b5fffff', '88618925b7fffff', '8860145949fffff', '8861892e41fffff', '886189219bfffff',
                                                 '886189251bfffff', '886016965dfffff', '8861892539fffff', '886189265bfffff', '8861892505fffff', '88618934a3fffff',
                                                   '8861892501fffff', '8860169681fffff')
and hour in ('17', '18', '19', '20', '21')
"""

#Load data into pandas table
df_apr_copy = pd.read_sql(q, connection)

In [ ]:
#Load and view dataset
df_apr = df_apr_copy.copy()
df_apr = df_apr[df_apr['pickup_location_hex_8'].isin(hyderabad_new_zone)]
df_apr = df_apr.merge(hexes, on = 'pickup_location_hex_8', how = 'left')
df_apr.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,captain_id,spd_fraud_flag,pickup_location_hex_8,Segment,Label
0,6442b79b8028a51b1e19f21e,NaN,88618925d7fffff,12.928399,77.592920,Bangalore,88618925d7fffff,88618924e7fffff,12.905614,77.585619,12.927035,77.593307,214939,21,2145,Auto,21:59,5,20230421,new,None,1682094050602,rider_busy,623ac99bb9a21e6231cbe1d3,None,88618925d7fffff,out-high,Holdout
1,6442b7c144636e16f07f9c77,NaN,88618925e9fffff,12.938040,77.628243,Bangalore,8861892513fffff,8861892537fffff,12.967276,77.641277,12.933443,77.632627,215017,21,2145,Auto,21:59,5,20230421,new,None,1682094048330,rider_busy,5d7cc15ed0286d106d8c81a9,None,8861892513fffff,in-high,Zone
2,6442b7c85028ee299f997a4c,NaN,88618925e3fffff,12.946871,77.618974,Bangalore,88618925e1fffff,88618925a5fffff,12.974552,77.609482,12.951653,77.624557,215024,21,2145,Auto,21:59,5,20230421,new,None,1682094051632,rider_busy,61271cf034b93d205731c911,None,88618925e1fffff,in-high,Zone
3,6442b6279199ef7231951e07,NaN,88618925c1fffff,12.924744,77.618340,Bangalore,88618925c7fffff,88618925e9fffff,12.939400,77.629547,12.935874,77.615479,214327,21,2130,Auto,21:59,5,20230421,new,None,1682093667679,rider_reject,611a1ceac304a5050c043b80,None,88618925c7fffff,in-high,Zone
4,6442b6a35028ee299f997822,NaN,88618925c7fffff,12.934337,77.616079,Bangalore,88618925c7fffff,88618925cdfffff,12.928712,77.629828,12.934815,77.611450,214531,21,2145,Auto,21:59,5,20230421,new,None,1682093938533,rider_reject,6102399ac1df0ac75cebdce8,None,88618925c7fffff,in-high,Zone


In [ ]:
#Create a count column
df_apr['count'] = 1

#Create a column to identify total pings and accepted ping
df_apr['total_pings'] = np.where(df_apr['event_type'].isin(['accepted', 'rider_busy', 'rider_reject']), 1, 0)
df_apr['accepted_pings'] = np.where(df_apr['event_type'].isin(['accepted']), 1, 0)
df_me = df_apr[(df_apr['order_status'] == 'dropped') & (df_apr['spd_fraud_flag'] != True)]

#Groupby on date and hour to get the APR and ME on an hourly basis
apr_hyderabad = df_apr.groupby(['yyyymmdd', 'hour', 'Segment', 'Label'])[['total_pings', 'accepted_pings']].sum().reset_index()
apr_hyderabad['APR'] = round(apr_hyderabad['accepted_pings']/apr_hyderabad['total_pings'], 4)
me_hyderabad = df_me.groupby(['yyyymmdd', 'hour', 'Segment', 'Label'])['order_id'].nunique().reset_index()
me_hyderabad = me_hyderabad.rename(columns={'order_id':'net_orders'})
apr_hyderabad = apr_hyderabad.merge(me_hyderabad, on = ['yyyymmdd', 'hour', 'Segment', 'Label'], how = 'left')
apr_hyderabad['ME'] = round(apr_hyderabad['total_pings']/apr_hyderabad['net_orders'], 2)

#Merge APR and ME with metrics_df
metrics_df = metrics_df.merge(apr_hyderabad, on = ['yyyymmdd', 'hour', 'Segment', 'Label'], how = 'left')
metrics_df.head()

,yyyymmdd,hour,Segment,Label,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total,FM,eta,number_of_captains,ttc_cobra,ttc_cobrm,ttc_ocara,tta,total_pings,accepted_pings,APR,net_orders,ME
0,20230421,17,balanced-high,Holdout,0.0969,0.2984,0.0000,0.4419,0.1628,0.0000,258,13.423963,2.899225,49.0,89.715,NA,347.593,11.460,3259,161,0.0494,114,28.59
1,20230421,17,balanced-high,Zone,0.0606,0.4545,0.0000,0.1919,0.2929,0.0000,99,64.619934,3.454545,33.0,102.802,NA,157.3835,37.074,1677,30,0.0179,19,88.26
2,20230421,17,balanced-low,Holdout,0.2113,0.3239,0.0000,0.2958,0.1690,0.0000,71,1.356786,3.647887,8.0,120.720,NA,148.46,9.057,776,46,0.0593,21,36.95
3,20230421,17,in-high,Holdout,0.1170,0.2591,0.0042,0.3412,0.2772,0.0014,718,0.890608,3.004172,63.5,90.416,32.572,238.538,11.828,9543,412,0.0432,244,39.11
4,20230421,17,in-high,Zone,0.0882,0.3594,0.0000,0.2601,0.2914,0.0000,1088,27.258625,2.719523,81.0,107.732,NA,222.206,17.916,14434,438,0.0303,282,51.18


In [ ]:
#Query data to calculate gsr
q = """
select 
        pickup_location_hex_8, yyyymmdd, hour,
        count(distinct order_id) as Gross_Orders,
        count(case when (time_diff >=20 or time_diff is NULL) then customer_id end) as GSR,
        count(distinct case when event_type in ('dropped') and spd_fraud_flag != True then order_id end) as net_orders,
        (case when count(distinct case when event_type in ('dropped') and spd_fraud_flag != True then order_id end) = 0 then 0 else
        sum(case when event_type in ('accepted', 'rider_busy', 'rider_reject') then 1 else 0 end)
        /cast(count(distinct case when event_type in ('dropped') and spd_fraud_flag != True then order_id end) as real) end) as ME
from
(
select
        *, 
        date_diff('minute',  LAG(FROM_UNIXTIME(CAST(epoch AS double) /1000)) OVER(PARTITION BY yyyymmdd, hour, city_name, customer_id, order_type, customer_location_hex_8
        ORDER BY yyyymmdd, customer_id, city_name, customer_location_hex_8, order_type, epoch), FROM_UNIXTIME(CAST(epoch AS double) /1000)) as time_diff,
        date_diff('minute',  LAG(FROM_UNIXTIME(CAST(epoch AS double) /1000)) OVER(PARTITION BY yyyymmdd, hour, city_name, customer_id, order_type, order_status, customer_location_hex_8, spd_fraud_flag 
        ORDER BY yyyymmdd, customer_id, city_name, customer_location_hex_8, order_type, epoch) , FROM_UNIXTIME(CAST(epoch AS double) /1000)) as time_diff_drop
from
(
select 
    order_id, city_name, order_type, cancel_reason, yyyymmdd, customer_id, epoch, substr(hhmmss,1,2) as hour,
    row_number() over (partition by order_id order by updated_epoch desc) as Rank_1,
    
    event_type, order_status, captain_id, pickup_cluster, customer_location_hex_8, spd_fraud_flag, pickup_location_hex_8

from
    orders.order_logs_immutable 
where
    yyyymmdd in ('20230424', '20230425', '20230426', '20230515', '20230516', '20230517')
    and service_obj_service_name IN ('Auto')
    and pickup_location_hex_8 in ('88618924bbfffff', '886189260bfffff', '8860145913fffff', '88618924b3fffff', '8860145823fffff', '8860169659fffff', '886014595dfffff',
 '8860145a07fffff', '8860145847fffff', '8860169607fffff', '8861892c81fffff', '88601451e1fffff', '8861892687fffff', '88618921e3fffff', '8861892187fffff',
   '88601451e5fffff', '8860145acdfffff', '88618920ebfffff', '886189203dfffff', '8860145a83fffff', '8860145a87fffff', '886189208dfffff', '8860169645fffff',
     '88601696dbfffff', '8861892debfffff', '8861892495fffff', '8860145125fffff', '8861892c3bfffff', '88618921c5fffff', '8860169621fffff', '8860169619fffff',
       '8860145aadfffff', '8860145921fffff', '88618934e3fffff', '886014c905fffff', '8860145907fffff', '88618921cdfffff', '8861892281fffff', '8860145101fffff',
         '8860145ab5fffff', '8861892117fffff', '8861893495fffff', '88601690c7fffff', '886016968dfffff', '8860145995fffff', '886189228dfffff', '886016944bfffff',
           '88618934b1fffff', '8861892f65fffff', '8860169687fffff', '8860169743fffff', '8860145115fffff', '8861892691fffff', '8860169655fffff', '8860145359fffff',
             '8861892461fffff', '8860145b3bfffff', '8861892427fffff', '8861892c4bfffff', '88618925e9fffff', '88618925c7fffff', '88618925c5fffff', '88618925c9fffff',
               '88618925c1fffff', '8861892437fffff', '8861892513fffff', '8861892c53fffff', '8861892e03fffff', '88618925edfffff', '8861892e33fffff',
                 '886189240bfffff', '8860145b43fffff', '8861892e5dfffff', '88618920adfffff', '88618925ebfffff', '8861892563fffff', '8861892cd3fffff',
                   '8861892721fffff', '8860145a59fffff', '8861892eb3fffff', '8861892e17fffff', '8861892e61fffff', '88618925e1fffff', '886189252bfffff',
                     '8861892e67fffff', '8860145b19fffff', '8860145a55fffff', '8861892603fffff', '8861892519fffff', '8861892511fffff', '8861892407fffff',
                       '8861892451fffff', '8861892e05fffff', '8861892717fffff', '8860145945fffff', '8861892ee7fffff', '8861892e45fffff', '8861892c09fffff',
                         '8861892517fffff', '88618925e3fffff', '88618926c1fffff', '8861892085fffff', '8861892625fffff', '88618920d3fffff', '886016975bfffff',
                           '8861892c69fffff', '886014534dfffff', '8860144b43fffff', '8860145acbfffff', '886014592bfffff', '8861892cb5fffff', '8861892ca9fffff',
                             '8861892da3fffff', '8860145133fffff', '8861892db9fffff', '88618926e1fffff', '8860145951fffff', '88601458c5fffff', '8860145831fffff',
                               '886014c925fffff', '8861892dc5fffff', '8861892dc7fffff', '88618926d1fffff', '88618935b7fffff', '8861892591fffff', '88618925d9fffff',
                                 '8861892425fffff', '8861892585fffff', '8861892e1bfffff', '8860145b1dfffff', '88601451adfffff', '8860145365fffff', '886014c9adfffff',
                                   '88618922b3fffff', '8860145ac9fffff', '8861892ca5fffff', '886014536bfffff', '88618935b1fffff', '8860145b37fffff',
                                     '8860145919fffff', '886014c9a9fffff', '886189269dfffff', '8860145a15fffff', '88618934b3fffff', '8860144b4dfffff',
                                       '88601451e7fffff', '8861892697fffff', '886014c923fffff', '8860145aedfffff', '886014c915fffff', '8860145341fffff',
                                         '8860145117fffff', '8861892293fffff', '8860169719fffff', '8860145363fffff', '88601459b5fffff', '8860145ae1fffff',
                                           '886014c92dfffff', '88601696e3fffff', '8861892c63fffff', '8860145113fffff', '8860144a6dfffff', '8860145315fffff',
                                             '8860144b55fffff', '886014c937fffff', '8860145abbfffff', '8860145b4bfffff', '8861892e91fffff', '8861892621fffff',
                                               '8860145b49fffff', '886189255bfffff', '8861892545fffff', '88618925d7fffff', '88618925cdfffff', '8861892e0dfffff', 
                                               '886189258dfffff', '88618925b5fffff', '88618925b7fffff', '8860145949fffff', '8861892e41fffff', '886189219bfffff',
                                                 '886189251bfffff', '886016965dfffff', '8861892539fffff', '886189265bfffff', '8861892505fffff', '88618934a3fffff',
                                                   '8861892501fffff', '8860169681fffff')
and hour in ('17', '18', '19', '20', '21')
))
group by 1,2,3
"""

df_gsr_copy = pd.read_sql(q, connection)

In [ ]:
#Create a copy of the dataset and view it
df_gsr = df_gsr_copy.copy()
df_gsr = df_gsr[df_gsr['pickup_location_hex_8'].isin(hyderabad_new_zone)]
df_gsr = df_gsr.merge(hexes, on = 'pickup_location_hex_8', how = 'left')
df_gsr.head()

,pickup_location_hex_8,yyyymmdd,hour,Gross_Orders,GSR,net_orders,ME,Segment,Label
0,8861892621fffff,20230512,17,57,31,6,64.50000,out-high,Holdout
1,8860169645fffff,20230422,17,6,6,1,25.00000,in-low,Holdout
2,8860145ac9fffff,20230513,19,11,5,0,0.00000,balanced-high,Holdout
3,8861892eb3fffff,20230513,18,11,6,0,0.00000,in-high,Holdout
4,88618925c7fffff,20230423,20,431,293,51,65.64706,in-high,Zone


In [ ]:
#Calculate GSR2Net
df_gsr_grouped = df_gsr.groupby(['yyyymmdd', 'hour', 'Segment', 'Label'])[['GSR', 'net_orders']].sum().reset_index()
df_gsr_grouped['GSR2Net'] = round(df_gsr_grouped['net_orders']/df_gsr_grouped['GSR'], 4)

#Merge GSR with metrics_df
metrics_df = metrics_df.merge(df_gsr_grouped, on = ['yyyymmdd', 'hour', 'Segment', 'Label'], how = 'left')
metrics_df.head()

,yyyymmdd,hour,Segment,Label,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total,FM,eta,number_of_captains,ttc_cobra,ttc_cobrm,ttc_ocara,tta,total_pings,accepted_pings,APR,net_orders_x,ME,GSR,net_orders_y,GSR2Net
0,20230421,17,balanced-high,Holdout,0.0969,0.2984,0.0000,0.4419,0.1628,0.0000,258,13.423963,2.899225,49.0,89.715,NA,347.593,11.460,3259,161,0.0494,114,28.59,208,114,0.5481
1,20230421,17,balanced-high,Zone,0.0606,0.4545,0.0000,0.1919,0.2929,0.0000,99,64.619934,3.454545,33.0,102.802,NA,157.3835,37.074,1677,30,0.0179,19,88.26,66,19,0.2879
2,20230421,17,balanced-low,Holdout,0.2113,0.3239,0.0000,0.2958,0.1690,0.0000,71,1.356786,3.647887,8.0,120.720,NA,148.46,9.057,776,46,0.0593,21,36.95,48,21,0.4375
3,20230421,17,in-high,Holdout,0.1170,0.2591,0.0042,0.3412,0.2772,0.0014,718,0.890608,3.004172,63.5,90.416,32.572,238.538,11.828,9543,412,0.0432,244,39.11,527,244,0.4630
4,20230421,17,in-high,Zone,0.0882,0.3594,0.0000,0.2601,0.2914,0.0000,1088,27.258625,2.719523,81.0,107.732,NA,222.206,17.916,14434,438,0.0303,282,51.18,762,282,0.3701


In [ ]:
#Query data from supply_login_servicelevel_kpi
q = """
select
        location as pickup_location_hex_8, yyyymmdd, substr(hhmm, 1, 2) as hour,
        sum(case when cast(status AS varchar) IN ('2','3','6','7','8','10') then duration end) as login_time,
        sum(case when cast(status AS varchar) IN ('2') then duration end) as idle_time
from 
(select 
        a.*, city, cluster
        
from 
        hive.datasets.supplycursory_history a 
        left join datasets.city_cluster_hex b 
        on a.location = b.hex_id and a.resolution = cast(b.resolution as varchar)
where 
        yyyymmdd in ('20230424', '20230425', '20230426', '20230515', '20230516', '20230517')
        and city in ('Bangalore')
        and a.resolution = '8' 
        and service in ('auto')
        and location in ('88618924bbfffff', '886189260bfffff', '8860145913fffff', '88618924b3fffff', '8860145823fffff', '8860169659fffff', '886014595dfffff',
 '8860145a07fffff', '8860145847fffff', '8860169607fffff', '8861892c81fffff', '88601451e1fffff', '8861892687fffff', '88618921e3fffff', '8861892187fffff',
   '88601451e5fffff', '8860145acdfffff', '88618920ebfffff', '886189203dfffff', '8860145a83fffff', '8860145a87fffff', '886189208dfffff', '8860169645fffff',
     '88601696dbfffff', '8861892debfffff', '8861892495fffff', '8860145125fffff', '8861892c3bfffff', '88618921c5fffff', '8860169621fffff', '8860169619fffff',
       '8860145aadfffff', '8860145921fffff', '88618934e3fffff', '886014c905fffff', '8860145907fffff', '88618921cdfffff', '8861892281fffff', '8860145101fffff',
         '8860145ab5fffff', '8861892117fffff', '8861893495fffff', '88601690c7fffff', '886016968dfffff', '8860145995fffff', '886189228dfffff', '886016944bfffff',
           '88618934b1fffff', '8861892f65fffff', '8860169687fffff', '8860169743fffff', '8860145115fffff', '8861892691fffff', '8860169655fffff', '8860145359fffff',
             '8861892461fffff', '8860145b3bfffff', '8861892427fffff', '8861892c4bfffff', '88618925e9fffff', '88618925c7fffff', '88618925c5fffff', '88618925c9fffff',
               '88618925c1fffff', '8861892437fffff', '8861892513fffff', '8861892c53fffff', '8861892e03fffff', '88618925edfffff', '8861892e33fffff',
                 '886189240bfffff', '8860145b43fffff', '8861892e5dfffff', '88618920adfffff', '88618925ebfffff', '8861892563fffff', '8861892cd3fffff',
                   '8861892721fffff', '8860145a59fffff', '8861892eb3fffff', '8861892e17fffff', '8861892e61fffff', '88618925e1fffff', '886189252bfffff',
                     '8861892e67fffff', '8860145b19fffff', '8860145a55fffff', '8861892603fffff', '8861892519fffff', '8861892511fffff', '8861892407fffff',
                       '8861892451fffff', '8861892e05fffff', '8861892717fffff', '8860145945fffff', '8861892ee7fffff', '8861892e45fffff', '8861892c09fffff',
                         '8861892517fffff', '88618925e3fffff', '88618926c1fffff', '8861892085fffff', '8861892625fffff', '88618920d3fffff', '886016975bfffff',
                           '8861892c69fffff', '886014534dfffff', '8860144b43fffff', '8860145acbfffff', '886014592bfffff', '8861892cb5fffff', '8861892ca9fffff',
                             '8861892da3fffff', '8860145133fffff', '8861892db9fffff', '88618926e1fffff', '8860145951fffff', '88601458c5fffff', '8860145831fffff',
                               '886014c925fffff', '8861892dc5fffff', '8861892dc7fffff', '88618926d1fffff', '88618935b7fffff', '8861892591fffff', '88618925d9fffff',
                                 '8861892425fffff', '8861892585fffff', '8861892e1bfffff', '8860145b1dfffff', '88601451adfffff', '8860145365fffff', '886014c9adfffff',
                                   '88618922b3fffff', '8860145ac9fffff', '8861892ca5fffff', '886014536bfffff', '88618935b1fffff', '8860145b37fffff',
                                     '8860145919fffff', '886014c9a9fffff', '886189269dfffff', '8860145a15fffff', '88618934b3fffff', '8860144b4dfffff',
                                       '88601451e7fffff', '8861892697fffff', '886014c923fffff', '8860145aedfffff', '886014c915fffff', '8860145341fffff',
                                         '8860145117fffff', '8861892293fffff', '8860169719fffff', '8860145363fffff', '88601459b5fffff', '8860145ae1fffff',
                                           '886014c92dfffff', '88601696e3fffff', '8861892c63fffff', '8860145113fffff', '8860144a6dfffff', '8860145315fffff',
                                             '8860144b55fffff', '886014c937fffff', '8860145abbfffff', '8860145b4bfffff', '8861892e91fffff', '8861892621fffff',
                                               '8860145b49fffff', '886189255bfffff', '8861892545fffff', '88618925d7fffff', '88618925cdfffff', '8861892e0dfffff', 
                                               '886189258dfffff', '88618925b5fffff', '88618925b7fffff', '8860145949fffff', '8861892e41fffff', '886189219bfffff',
                                                 '886189251bfffff', '886016965dfffff', '8861892539fffff', '886189265bfffff', '8861892505fffff', '88618934a3fffff',
                                                   '8861892501fffff', '8860169681fffff')
and substr(hhmm, 1, 2) in ('17', '18', '19', '20', '21')
)
group by 1,2,3
"""

#Load data into pandas table
df_tse_copy = pd.read_sql(q, connection)

In [ ]:
#Create a copy of the df and view it
df_tse = df_tse_copy.copy()
df_tse = df_tse[df_tse['pickup_location_hex_8'].isin(hyderabad_new_zone)]
df_tse = df_tse.merge(hexes, on = 'pickup_location_hex_8', how = 'left')
df_tse.head()

,pickup_location_hex_8,yyyymmdd,hour,login_time,idle_time,Segment,Label
0,886189208dfffff,20230514,18,2490.0,100.0,in-low,Holdout
1,8861892e67fffff,20230513,19,8039.0,3149.0,in-high,Holdout
2,8861892c4bfffff,20230421,17,41573.0,14849.0,in-high,Holdout
3,8861892c4bfffff,20230514,18,12803.0,954.0,in-high,Holdout
4,8861892e67fffff,20230514,18,5992.0,149.0,in-high,Holdout


In [ ]:
#Calculate time_spent_earning
df_tse_grouped = df_tse.groupby(['yyyymmdd', 'hour', 'Segment', 'Label'])[['login_time', 'idle_time']].sum().reset_index()
df_tse_grouped['time_spent_earning'] = round((df_tse_grouped['login_time'] - df_tse_grouped['idle_time'])/df_tse_grouped['login_time'], 4)
df_tse_grouped.head()

,yyyymmdd,hour,Segment,Label,login_time,idle_time,time_spent_earning
0,20230421,17,balanced-high,Holdout,415055.0,135846.0,0.6727
1,20230421,17,balanced-high,Zone,124658.0,27231.0,0.7816
2,20230421,17,balanced-low,Holdout,181515.0,127322.0,0.2986
3,20230421,17,in-high,Holdout,1391947.0,524434.0,0.6232
4,20230421,17,in-high,Zone,1128646.0,102572.0,0.9091


In [ ]:
#Merge GSR with metrics_df
metrics_df = metrics_df.merge(df_tse_grouped, on = ['yyyymmdd', 'hour', 'Segment', 'Label'], how = 'left')
metrics_df.head()

,yyyymmdd,hour,Segment,Label,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total,FM,eta,number_of_captains,ttc_cobra,ttc_cobrm,ttc_ocara,tta,total_pings,accepted_pings,APR,net_orders_x,ME,GSR,net_orders_y,GSR2Net,login_time,idle_time,time_spent_earning
0,20230421,17,balanced-high,Holdout,0.0969,0.2984,0.0000,0.4419,0.1628,0.0000,258,13.423963,2.899225,49.0,89.715,NA,347.593,11.460,3259,161,0.0494,114,28.59,208,114,0.5481,415055.0,135846.0,0.6727
1,20230421,17,balanced-high,Zone,0.0606,0.4545,0.0000,0.1919,0.2929,0.0000,99,64.619934,3.454545,33.0,102.802,NA,157.3835,37.074,1677,30,0.0179,19,88.26,66,19,0.2879,124658.0,27231.0,0.7816
2,20230421,17,balanced-low,Holdout,0.2113,0.3239,0.0000,0.2958,0.1690,0.0000,71,1.356786,3.647887,8.0,120.720,NA,148.46,9.057,776,46,0.0593,21,36.95,48,21,0.4375,181515.0,127322.0,0.2986
3,20230421,17,in-high,Holdout,0.1170,0.2591,0.0042,0.3412,0.2772,0.0014,718,0.890608,3.004172,63.5,90.416,32.572,238.538,11.828,9543,412,0.0432,244,39.11,527,244,0.4630,1391947.0,524434.0,0.6232
4,20230421,17,in-high,Zone,0.0882,0.3594,0.0000,0.2601,0.2914,0.0000,1088,27.258625,2.719523,81.0,107.732,NA,222.206,17.916,14434,438,0.0303,282,51.18,762,282,0.3701,1128646.0,102572.0,0.9091


In [ ]:
#Create a copy of the dataset and view it
df_captains_mapped = df_copy.copy()
df_captains_mapped = df_captains_mapped[df_captains_mapped['pickup_location_hex_8'].isin(hyderabad_new_zone)]
df_captains_mapped = df_captains_mapped.merge(hexes, on = 'pickup_location_hex_8', how = 'left')
df_captains_mapped.head()

,order_id,accept_to_pickup_distance,city_name,hhmmss,hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8,Segment,Label
0,64452e49bc576a70d21a5ab8,NaN,Bangalore,184033,18,Auto,18:59,7,20230423,expired,None,2.0,"[""60f6801bd417d4450f7bf96d""]",886189240bfffff,in-high,Holdout
1,64452e763ec61546f395b40f,1.707,Bangalore,184118,18,Auto,18:59,7,20230423,customerCancelled,Wrong pickup location,7.0,"[""616ab1d8fbce6611f99a1d7c"",""63f3396ef1e0507c9...",88618925c1fffff,in-high,Zone
2,64452ecf7c88b1060e929652,NaN,Bangalore,184247,18,Auto,18:59,7,20230423,customerCancelled,order cancelled before rider accepted,5.0,"[""6253ab54cc5c39a6c8b43e6c"",""610384aa342a669bd...",8861892545fffff,out-high,Holdout
3,64452fd0dddb855c79ca66b0,NaN,Bangalore,184704,18,Auto,18:59,7,20230423,expired,None,6.0,"[""6430f71bf6c53c6b7dd51557"",""6426872cd0cc052c2...",886189240bfffff,in-high,Holdout
4,64452ff83c928b66bd6e5dd8,1.589,Bangalore,184744,18,Auto,18:59,7,20230423,dropped,None,5.0,"[""62e0d55a4a9e686fbbff72c9"",""640984555516ce08a...",8860145a83fffff,in-low,Holdout


In [ ]:
#Get the number of captains mapped
df_captains_mapped['map_riders'] = df_captains_mapped['map_riders'].apply(lambda x: json.loads(x))
df_captains_mapped_grouped = df_captains_mapped.groupby(['yyyymmdd', 'hour', 'Segment', 'Label'])['map_riders'].apply(list).reset_index()
df_captains_mapped_grouped['map_riders'] = df_captains_mapped_grouped['map_riders'].apply(lambda x: list(set([item for sublist in x for item in sublist])))
master_list = []
for x in df_captains_mapped_grouped['map_riders']:
    captains_list = []
    for captain in x:
        if captain in captains_list:
            continue
        else:
            captains_list.append(captain)
    master_list.append(captains_list)
df_captains_mapped_grouped['map_riders'] = master_list
df_captains_mapped_grouped['number_of_mapped_captains'] = df_captains_mapped_grouped['map_riders'].apply(lambda x: len(x))
metrics_df = metrics_df.merge(df_captains_mapped_grouped[['yyyymmdd', 'hour', 'Segment', 'Label', 'number_of_mapped_captains']], 
on = ['yyyymmdd', 'hour', 'Segment', 'Label'], how = 'left')
metrics_df.head()

,yyyymmdd,hour,Segment,Label,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total,FM,eta,number_of_captains,ttc_cobra,ttc_cobrm,ttc_ocara,tta,total_pings,accepted_pings,APR,net_orders_x,ME,GSR,net_orders_y,GSR2Net,login_time,idle_time,time_spent_earning,number_of_mapped_captains
0,20230421,17,balanced-high,Holdout,0.0969,0.2984,0.0000,0.4419,0.1628,0.0000,258,13.423963,2.899225,49.0,89.715,NA,347.593,11.460,3259,161,0.0494,114,28.59,208,114,0.5481,415055.0,135846.0,0.6727,1414
1,20230421,17,balanced-high,Zone,0.0606,0.4545,0.0000,0.1919,0.2929,0.0000,99,64.619934,3.454545,33.0,102.802,NA,157.3835,37.074,1677,30,0.0179,19,88.26,66,19,0.2879,124658.0,27231.0,0.7816,710
2,20230421,17,balanced-low,Holdout,0.2113,0.3239,0.0000,0.2958,0.1690,0.0000,71,1.356786,3.647887,8.0,120.720,NA,148.46,9.057,776,46,0.0593,21,36.95,48,21,0.4375,181515.0,127322.0,0.2986,330
3,20230421,17,in-high,Holdout,0.1170,0.2591,0.0042,0.3412,0.2772,0.0014,718,0.890608,3.004172,63.5,90.416,32.572,238.538,11.828,9543,412,0.0432,244,39.11,527,244,0.4630,1391947.0,524434.0,0.6232,3573
4,20230421,17,in-high,Zone,0.0882,0.3594,0.0000,0.2601,0.2914,0.0000,1088,27.258625,2.719523,81.0,107.732,NA,222.206,17.916,14434,438,0.0303,282,51.18,762,282,0.3701,1128646.0,102572.0,0.9091,1199


In [ ]:
metrics_df.to_csv('metrics_holdout_hsr_upper_new.csv', index = False)